In [53]:
import os
import nibabel as nib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.utils import Sequence, to_categorical

In [54]:
SCAN_DIR = '/kaggle/input/adni-processed/ADNI1_Processed'
CSV_PATH = '/kaggle/input/scan-labels/ADNI1_Complete_1Yr_1.5T_6_20_2025.csv' 

In [55]:
IMG_SIZE = 224
NUM_SLICES = 40
BATCH_SIZE = 8
EPOCHS = 5

# Load CSV and create label map
df = pd.read_csv(CSV_PATH)
df['Subject'] = df['Subject'].astype(str).str.strip()
df['Group'] = df['Group'].astype(str).str.strip()
label_map = dict(zip(df['Subject'], df['Group']))

In [56]:
scan_paths, scan_labels = [], []
for root, _, files in os.walk(SCAN_DIR):
    for file in files:
        if file.endswith('.nii') or file.endswith('.nii.gz'):
            full_path = os.path.join(root, file)
            parts = full_path.split(os.sep)
            try:
                subj_id = parts[5]  # e.g., 133_S_0913 from /kaggle/input/adni-processed/ADNI1_Processed/133_S_0913/Ixxxx/file.nii
                if subj_id in label_map:
                    scan_paths.append(full_path)
                    scan_labels.append(label_map[subj_id])
            except IndexError:
                continue

print(f"✅ Total scans with labels: {len(scan_paths)}")

✅ Total scans with labels: 459


In [57]:
# Encode labels
le = LabelEncoder()
encoded_labels = le.fit_transform(scan_labels)

# Train-test split
train_paths, test_paths, y_train, y_test = train_test_split(
    scan_paths, encoded_labels, test_size=0.2, random_state=42, stratify=encoded_labels
)

In [58]:
def calculate_entropy(slice_2d):
    histogram, _ = np.histogram(slice_2d, bins=256)
    histogram = histogram / np.sum(histogram) + 1e-8
    return -np.sum(histogram * np.log2(histogram))

def extract_top_slices(scan_path, num_slices=NUM_SLICES):
    scan = nib.load(scan_path).get_fdata()
    slice_entropies = [(i, calculate_entropy(scan[:, :, i])) for i in range(scan.shape[2])]
    slice_entropies.sort(key=lambda x: x[1], reverse=True)
    top_indices = [i for i, _ in slice_entropies[:num_slices]]
    slices = [scan[:, :, i] for i in sorted(top_indices)]
    slices = [np.stack([s]*3, axis=-1) for s in slices]  # Convert to RGB
    resized = [tf.image.resize(s, (IMG_SIZE, IMG_SIZE)).numpy() for s in slices]
    return np.stack(resized)

In [59]:
class ADNISequence(Sequence):
    def __init__(self, paths, labels, batch_size=BATCH_SIZE, is_train=True):
        self.paths = paths
        self.labels = labels
        self.batch_size = batch_size
        self.is_train = is_train

    def __len__(self):
        return int(np.ceil(len(self.paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_paths = self.paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        X, y = [], []
        for path, label in zip(batch_paths, batch_labels):
            try:
                slices = extract_top_slices(path)
                X.append(slices)
                y.extend([label] * slices.shape[0])
            except Exception as e:
                print(f"Skipping {path}: {e}")
        X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
        y = to_categorical(np.array(y), num_classes=len(le.classes_))
        return X, y

In [60]:
def build_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=3):
    base_model = VGG19(include_top=False, weights='imagenet', input_tensor=Input(shape=input_shape))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [61]:
train_gen = ADNISequence(train_paths, y_train)
test_gen = ADNISequence(test_paths, y_test, is_train=False)
steps_per_epoch = len(train_gen)
validation_steps = len(test_gen)

# Build and train
model = build_model(num_classes=len(le.classes_))
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10,
    steps_per_epoch=len(train_gen),
    validation_steps=len(test_gen)
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_24']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


46/46 ━━━━━━━━━━━━━━━━━━━━ 338s 7s/step - accuracy: 0.3488 - loss: 1.6967 - val_accuracy: 0.4122 - val_loss: 1.1403
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 327s 7s/step - accuracy: 0.4131 - loss: 1.1644 - val_accuracy: 0.4834 - val_loss: 1.1073
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 328s 7s/step - accuracy: 0.4478 - loss: 1.1359 - val_accuracy: 0.4560 - val_loss: 1.0713
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 325s 7s/step - accuracy: 0.4894 - loss: 1.0394 - val_accuracy: 0.3935 - val_loss: 1.1051
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 327s 7s/step - accuracy: 0.4396 - loss: 1.0722 - val_accuracy: 0.4601 - val_loss: 1.0695
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 332s 7s/step - accuracy: 0.4533 - loss: 1.0729 - val_accuracy: 0.4834 - val_loss: 1.1187
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 329s 7s/step - accuracy: 0.5398 - loss: 0.9847 - val_accuracy: 0.4818 - val_loss: 1.0341
Epoch 8/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 326s 7s/step - accuracy: 0.5058 - loss: 0.9980 - val_accuracy: 0.3459 - val_loss: 1.

In [62]:
y_pred, y_true = [], []
for X_batch, y_batch in test_gen:
    preds = model.predict(X_batch, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(np.argmax(y_batch, axis=1))

print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=le.classes_))


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_24']
Received: inputs=Tensor(shape=(32, 224, 224, 3))
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


UnboundLocalError: cannot access local variable 'batch_outputs' where it is not associated with a value